# Project: TMDB Movie Data Analysis

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

In this project we'll be analyzing data associated with movies released between 1960-2015. In particular we'll be interested in finding trends among several variables, including: average rating by movie genre, properties associated with high-revenue movies, whether the month a movie is released has an effect on its revenue, and whether movie length has an affect on ratings.  

The dataset and documentation  can be found here: 

[Movie Data](https://d17h27t6h515a5.cloudfront.net/topher/2017/October/59dd1c4c_tmdb-movies/tmdb-movies.csv) 

[Documentation](https://www.kaggle.com/tmdb/tmdb-movie-metadata/kernels)

### 1. Importing the required libraries for data analysis

Below are the libraries we need for exploratory data analysis in this project

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

<a id='wrangling'></a>
## Data Wrangling

### 2.  We load our dataset and print a few rows to get an idea of the data we're working with.

In [10]:
df_movies = pd.read_csv('tmdb_movies.csv')
df_movies.head(3)

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,5562,6.5,2015,137999939.3,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,6185,7.1,2015,137999939.3,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,2480,6.3,2015,101199955.5,2.716190e+08


We have 21 columns of data to work with. Before we move on, here is a description of the columns to make it clear what each one represents:

| Column               | Description                                                                                                                                                                         |
|----------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| id                   | unique identifier for each movie                                                                                                                                                    |
| imdb_id              | unique identifier in imdb for each movie                                                                                                                                            |
| popularity           | [popularity of movie on the tmdb website](https://developers.themoviedb.org/3/getting-started/popularity) |
| budget               | movie budget when it was released                                                                                                                                                   |
| revenue              | movie revenue                                                                                                                                                                       |
| original_title       | movie title                                                                                                                                                                         |
| cast                 | movie cast                                                                                                                                                                          |
| homepage             | movie website                                                                                                                                                                       |
| director             | movie director                                                                                                                                                                      |
| tagline              | movie catchphrase                                                                                                                                                                   |
| overview             | overview of movie plot                                                                                                                                                              |
| runtime              | movie length in minutes                                                                                                                                                             |
| genres               | list of genres the movie is included in                                                                                                                                             |
| production_companies | production companies for the movie                                                                                                                                                  |
| release_date         | date the movie was released                                                                                                                                                         |
| vote_count           | number of votes for the movie on tmdb                                                                                                                                               |
| vote_average         | average movie rating between 1-10                                                                                                                                                   |
| release_year         | year movie was released                                                                                                                                                             |
| budget_adj           | adjusted budget in terms of 2010 dollars, accounting for inflation over time                                                                                                        |
| revenue_adj          | adjusted revenue in terms of 2010 dollars, accounting for inflation over time                                                                                                       |

> **Tip**: You should _not_ perform too many operations in each cell. Create cells freely to explore your data. One option that you can take with this project is to do a lot of explorations in an initial notebook. These don't have to be organized, but make sure you use enough comments to understand the purpose of each code cell. Then, after you're done with your analysis, create a duplicate notebook where you will trim the excess and organize your steps so that you have a flowing, cohesive report.

> **Tip**: Make sure that you keep your reader informed on the steps that you are taking in your investigation. Follow every code cell, or every set of related code cells, with a markdown cell to describe to the reader what was found in the preceding cell(s). Try to make it so that the reader can then understand what they will be seeing in the following cell(s).

### Data Cleaning (Replace this with more specific notes!)

In [11]:
# After discussing the structure of the data and any problems that need to be
#   cleaned, perform those cleaning steps in the second part of this section.


<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Research Question 1 (Replace this header name!)

In [12]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Research Question 2  (Replace this header name!)

In [13]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!